In [1]:
import pandas as pd
import os

In [2]:
path = f'{os.getcwd()}/PDBs'

In [3]:
df = pd.read_csv(f'{path}/APD3/APD3_2023_02_02.csv')

In [4]:
def multi_replace(text, replaces) -> (str, dict):
    for key in replaces.keys():
        text = text.replace(key, replaces[key])
    return text

In [6]:
# Fix type error in line 2745
df.loc[2745, 'Reference'] = 'Not Found'

In [9]:
# Get DOIs from DataFrame
doi = {
    'Index': [],
    'DOI': []
}

replaces = {
    'doi: ': 'doi:',
    '.Pub': ' ',
    'doi:': 'doi: ',
    'doi.org/': 'doi: ',
    '.Abs': ' '
}

for i in range(0, len(df['Reference'])):
    for ref_info in df['Reference'][i].split('. '):
        if 'doi' in ref_info:
            ref_info = multi_replace(ref_info, replaces)
            doi['Index'].append(i)
            doi['DOI'].append("https://doi.org/".join(ref_info.split('doi:')[1].split(' ')[1]))

In [36]:
references = {
    'Name': [],
    'Count': []
}

replaces = {
    'Pub-Med': 'PubMed',
    ' PubMed': 'PubMed',
    ' Pub-Med': 'PubMed',
    ' OnlineJournal': 'OnlineJournal',
    ' Publiser': 'Publiser',
    ' SciDirect': 'ScienceDirect',
    '1': ' '
}

for ref in df['Reference']:
    # Get website name
    ref_info = [x for x in ref.split('.') if x != '']
    ref_name = ref_info[len(ref_info)-1]
    
    # Execute replaces
    if not ' ' in ref_name or ref_name in replaces.keys():
        ref_name = multi_replace(ref_name, replaces)
        
    # Verify names what don't have references
    if (not 'Submit' in ref_name and ' ' in ref_name) or len(ref_name) < 5:
        ref_name = 'Not Found'
    
    # Verify if name already stays at list and increment count
    exists = [x for x in references['Name'] if x in ref_name]
    if not exists:
        references['Name'].append(ref_name)
        references['Count'].append(1)
    else:
        for i in range(0, len(references['Name'])-1):
            if references['Name'][i] == ref_name:
                references['Count'][i] += 1

In [37]:
# Count of reference sites
for i in range(0, len(references['Count'])):
    print(references['Name'][i])
    print(references['Count'][i])

PubMed
2622
OnlineJournal
1
Not Found
900
Submitted JUL-2001 to the SWISS-PROT data bank
1
Submitted JUL-2002 to the SWISS-PROT 
1
Submitted JUL-2001 to the EMBL GenBank DDBJ databases
1
Publiser
2
Submitted APR-2002 to the SWISS-PROT data bank
1
Submitted JUN-1998 to the EMBL GenBank DDBJ databases
1
Submitted MAR-1995 to the EMBL GenBank DDBJ databases
1
ScienceDirect
2
Submitted DEC-1999 to the SWISS-PROT data bank
1
GenBank
1
